In [70]:
import pandas as pd
import numpy as np
import forecast

In [71]:
# Загрузим данные
predictions = pd.read_csv('forecasts/BTCUSD.csv')
candles_df = forecast.get_df('BTCUSD', '2024-01-02 00:00', '1T')
predictions['last_candle_time'] = pd.to_datetime(predictions['last_candle_time'])

# Функция для расчета фактических движений цены вверх
def calculate_actual_movements(predictions, candles_df):
    actual_movements = []
    
    for idx, row in predictions.iterrows():
        last_candle_time = row['last_candle_time']
        p_up = row['p_up']
        
        # Найдем цену закрытия в момент прогноза
        close_price = candles_df.loc[last_candle_time]['close']
        
        # Найдем цену открытия через пять минут
        future_time = last_candle_time + pd.Timedelta(minutes=5)
        open_price = candles_df.loc[future_time]['open']
        
        # Сравним цены
        if open_price > close_price:
            actual_movements.append(1)
        else:
            actual_movements.append(0)
    
    return actual_movements

# Рассчитаем фактические движения
predictions['actual_up'] = calculate_actual_movements(predictions, candles_df)

In [72]:
# Получаем уникальные значения вероятности p_up
unique_p_up_values = predictions['p_up'].unique()

# Словарь для хранения результатов
p_up_to_actual_probability = {}

# Рассчитываем фактическую вероятность для каждого уникального значения p_up
for p_up in unique_p_up_values:
    # Фильтруем строки с текущим значением p_up
    filtered_predictions = predictions[predictions['p_up'] == p_up]
    
    # Рассчитываем фактическую вероятность роста для текущего значения p_up
    actual_probability = sum(filtered_predictions['actual_up']) / len(filtered_predictions['actual_up'])
    sample_size = len(filtered_predictions['actual_up'])
    p_up_to_actual_probability[p_up] = (actual_probability, sample_size)

# Выводим результаты
for forecasted_p_up, (actual_p_up, sample_size) in p_up_to_actual_probability.items():
    print(f'Forecasted p_up: {forecasted_p_up}%, Actual p_up: {actual_p_up * 100:.2f}%, Sample size: {sample_size}')


Forecasted p_up: 45%, Actual p_up: 63.33%, Sample size: 30
Forecasted p_up: 60%, Actual p_up: 45.10%, Sample size: 153
Forecasted p_up: 65%, Actual p_up: 62.50%, Sample size: 32
Forecasted p_up: 50%, Actual p_up: 64.29%, Sample size: 14
Forecasted p_up: 35%, Actual p_up: 71.43%, Sample size: 14
Forecasted p_up: 30%, Actual p_up: 50.00%, Sample size: 36
Forecasted p_up: 55%, Actual p_up: 43.28%, Sample size: 67
Forecasted p_up: 40%, Actual p_up: 47.06%, Sample size: 51
Forecasted p_up: 70%, Actual p_up: 56.41%, Sample size: 39
Forecasted p_up: 25%, Actual p_up: 50.00%, Sample size: 2
Forecasted p_up: 20%, Actual p_up: 25.00%, Sample size: 4
